In [3]:
import pandas as pd
# import modin.pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from plotly.graph_objs.layout import YAxis,XAxis
import pathlib
import numpy as np
import pyarrow as pa
import h5py
import sqlite3
import glob
import os
import re
from tqdm import tqdm
from pynsys import nsys_constants

tqdm.pandas()


COMPRESSION_KERNELS = ['snap_kernel', 'lz_compression_kernel', 'lz4CompressBatchKernel', 'compress_kernel', 'batch_encoder_kernel', 'cascaded_compression_kernel', 'lz_compress_longest_matches', 'lz_compress_greedy_hash', 'huffman_encode', 'gdeflate_encode']
DECOMPRESSION_KERNELS = ['unsnap_kernel', 'decompression_kernel', 'lz4DecompressBatchKernel', 'decompress_kernel', 'batch_decoder_kernel', 'cascaded_decompression_kernel_type_check', 'inflate_kernel', 'gdeflateDecompress']

CATEGORY_COMPRESSION='compress'
CATEGORY_DECOMPRESSION='decompress'
CATEGORY_OTHER='other'

In [4]:
# Make df from hdf/h5
# input_file_hdf = "../../../data/decompression/nvcomp_results_2024-03-21-123552/output_zstd_8gb_dickens_131072threads.h5"
# input_file_hdf = "../data/compare_gpu/rtx3050ti/output_cascaded_500mb_xml_2048threads.h5"
# input_file_hdf = "../data/compare_gpu/rtx2080ti/output_cascaded_500mb_xml_2048threads.h5"
input_file_hdf = "../data/compare_gpu_exact/a100/output_cascaded_1gb_xml_2048threads.h5"

# input_file_hdf = "../data/gohan/lzbench/profile/nvcomp_cascaded_nsys_results_2024-03-19-121204.h5"
# file_size = 8589934592
file_size = 524288000

hdf_file = h5py.File(input_file_hdf)

gpu_metric_df = pd.DataFrame(np.array(hdf_file[nsys_constants.TABLE_GPU_METRICS]))
kernel_event_df = pd.DataFrame(np.array(hdf_file[nsys_constants.TABLE_CUPTI_ACTIVITY_KIND_KERNEL]))
strings_df = pd.DataFrame(np.array(hdf_file[nsys_constants.TABLE_STRING_IDS]))
target_info_gpu_metrics_df = pd.DataFrame(np.array(hdf_file[nsys_constants.TABLE_TARGET_INFO_GPU_METRICS]))

# print(target_info_gpu_metrics_df.to_string())
                       
gpu_metric_df = gpu_metric_df.sort_values(by=['timestamp'])
kernel_event_df = kernel_event_df.sort_values(by=['start'])


In [5]:
# def human_format(num):
#     magnitude = 0
#     while abs(num) >= 1024:
#         magnitude += 1
#         num /= 1024
#     return '{}{}B'.format('{}'.format(num).rstrip('0').rstrip('.'), ['', 'k', 'M', 'G', 'T'][magnitude])

# filesize = (8 * 1024 * 1024 * 1024)
# results_df = pd.read_csv("../../../data/decompression/nvcomp_results_2024-03-21-123552/results.csv")
# results_df['chunksize_Bytes'] = filesize / results_df['chunks']
# results_df['chunksize_Bytes_formatted'] = results_df['chunksize_Bytes'].apply(human_format)
# results_df['chunks_formatted'] = results_df['chunks'].apply(human_format)
# results_df['chunks_combined_formatted'] = results_df['chunks'].apply(lambda x : f"{human_format(x)} chunks of {human_format(filesize / x)}B")

In [6]:
def get_string(id) -> str:
    return str(strings_df.loc[strings_df['id'] == id, 'value'].values[0], encoding='utf-8')

def decode_bytes(byte_str):
    return byte_str.decode('utf-8')

def get_gpu_metric_id(name) -> int:
    metric_ids = target_info_gpu_metrics_df[target_info_gpu_metrics_df['metricName'].apply(decode_bytes).str.contains(name, case=False)]
    if len(metric_ids) > 0:
        return int(metric_ids['metricId'].values[0])
    print('Did not find metric id for ' + str(name))
    return 17

def get_utilizations_of_span(begin, end):
    # Select utilization data
    utilisation_metric_id = get_gpu_metric_id(nsys_constants.METRIC_COMPUTE_WARPS)
    compute_metrics = gpu_metric_df.loc[gpu_metric_df['metricId'] == utilisation_metric_id]
    return compute_metrics[compute_metrics['timestamp'].between(begin, end, inclusive="both")]

def get_avg_utilization_of_span(begin, end) -> float:
    utilizations = get_utilizations_of_span(begin, end)
    if len(utilizations) < 1:
        print('Waning: No utilization data found in provided range')
        return -1
    if len(utilizations) == 1:
        return utilizations['value'].iloc[0]
    # Get the time difference between all metric timestamps
    utilizations['metric_diff'] = utilizations['timestamp'].diff()
    # Remove broken data
    utilizations = utilizations.dropna(subset=['metric_diff', 'value'])
    # Total time difference between all metric points in the span
    time_diff_sum = utilizations['metric_diff'].sum()
    if time_diff_sum == 0:
        print("No time difference between metrics")
        return -1
    # Weighted average of each metric and their duration
    utilization = (utilizations['metric_diff'] * utilizations['value']).sum() / time_diff_sum
    
    return utilization

# print(str(get_utilizations_of_span(2097867769.0, 2097917433.0)))
# print('hello')
kernel_event_df['utilization'] = kernel_event_df.progress_apply(lambda row: get_avg_utilization_of_span(row['start'], row['end']), axis=1)
kernel_event_df['name'] = kernel_event_df['shortName'].progress_apply(get_string)
kernel_event_df['chunk_size'] = (file_size / kernel_event_df['gridX']).astype(int)
kernel_event_df['duration'] = kernel_event_df['end'] - kernel_event_df['start']
# print(kernel_event_df.to_string())

100%|██████████| 16/16 [00:00<00:00, 239.12it/s]


Did not find metric id for Compute Warps In Flight
Did not find metric id for Compute Warps In Flight
Did not find metric id for Compute Warps In Flight
Did not find metric id for Compute Warps In Flight
Did not find metric id for Compute Warps In Flight
Waning: No utilization data found in provided range
Did not find metric id for Compute Warps In Flight
Waning: No utilization data found in provided range
Did not find metric id for Compute Warps In Flight
Did not find metric id for Compute Warps In Flight
Waning: No utilization data found in provided range
Did not find metric id for Compute Warps In Flight
Did not find metric id for Compute Warps In Flight
Did not find metric id for Compute Warps In Flight
Did not find metric id for Compute Warps In Flight
Did not find metric id for Compute Warps In Flight
Waning: No utilization data found in provided range
Did not find metric id for Compute Warps In Flight
Waning: No utilization data found in provided range
Did not find metric id for

100%|██████████| 16/16 [00:00<00:00, 9301.30it/s]


In [7]:
utilisation_metric_id = get_gpu_metric_id(nsys_constants.METRIC_COMPUTE_WARPS)

compute_metrics = gpu_metric_df.loc[gpu_metric_df['metricId'] == utilisation_metric_id]


fig1 = px.scatter(compute_metrics, x="timestamp", y="value")

fig1.update_layout(title='Utilization over time',
                  xaxis_title='Timestamp (ns)',
                  yaxis_title='Compute in flight')

fig1.update_traces(marker=dict(size=2))
fig1.show()


Did not find metric id for Compute Warps In Flight


In [8]:
# Get timestamps of start and end of compression
kernel_events = kernel_event_df

kernel_events = kernel_events[kernel_events['utilization'] != -1]



fig2 = px.scatter(kernel_event_df, y="utilization", x="gridX", color="name", size="duration", log_x=True, size_max=40)
fig2.update_layout(title='Utilization per grid size',
                  xaxis_title='gridX',
                  yaxis_title='Average utilization')
fig2.update_traces(marker_sizemin=4, selector=dict(type='scatter'))

fig2.update_layout(scattermode="group")
fig2.show()



In [9]:
# gridX = filesize / chunk size
# chunk size = filesize / gridX

fig = px.scatter(kernel_event_df, y="utilization", x="chunk_size", color="name", size="duration", log_x=True, size_max=40)
fig.update_traces(marker_sizemin=4, selector=dict(type='scatter'))
fig.update_layout(scattermode="group")
fig.show()


In [10]:
# Collect all kernels that work together to do 1 (de)compression for a file at a chunk size
time_splitter_ns = 50_000

def weighted_mean(df):
    return (df['utilization'] * df['duration']).sum() / df['duration'].sum()

# Calculate time between two kernel events
kernel_event_df['time_since_previous'] = kernel_event_df['start'] - kernel_event_df['end'].shift(1)
kernel_event_df['group'] = kernel_event_df['time_since_previous'].gt(time_splitter_ns).cumsum()
print(kernel_event_df.to_string())

def categorize_group(group):
    # any(string in COMPRESSION_KERNELS for string in group['name'].values)
    if any(string in COMPRESSION_KERNELS for string in group['name'].values):
        return CATEGORY_COMPRESSION
    elif any(string in DECOMPRESSION_KERNELS for string in group['name'].values):
        return CATEGORY_DECOMPRESSION
    else:
        return CATEGORY_OTHER


# Apply the function to each group
groups_durations = kernel_event_df.groupby('group')['duration'].sum().rename('duration').reset_index()
groups_categories = kernel_event_df.groupby('group').apply(categorize_group).rename('category')
groups_utilizations = kernel_event_df.groupby('group').apply(weighted_mean).rename('utilization')
groups_num_kernels = kernel_event_df.groupby('group').size().rename('num_kernels')

group_data = pd.concat([groups_categories, groups_durations, groups_utilizations, groups_num_kernels], axis=1)
print(group_data)


# print(grouped_data)
fig4 = px.bar(group_data, x=['category', 'group'], y='utilization', title='Utilization per kernel', text='category')
fig4.show()

fig5 = px.bar(group_data, x=['category', 'group'], y='duration', title='Duration per kernel', text='category')
fig5.show()



         start         end  deviceId  contextId  streamId  correlationId        globalPid  demangledName  shortName  mangledName  launchType  cacheConfig  registersPerThread    gridX  gridY  gridZ  blockX  blockY  blockZ  staticSharedMemory  dynamicSharedMemory  localMemoryPerThread  localMemoryTotal  gridId  sharedMemoryExecuted  graphNodeId  sharedMemoryLimitConfig  utilization                                      name  chunk_size  duration  time_since_previous  group
0   2808094607  2810013292         3          1         7            940  332597435564032           2517       2518         2516           1            1                  16  2097152      1      1     256       1       1                   0                    0                     0         127401984       1                 32768            0                        0    38.933541                             _kernel_agent         250   1918685                  NaN      0
1   2985164440  2986720789         3          1   

/var/folders/5z/1fjczghs4x101sqrk1z655tc0000gn/T/ipykernel_18309/1906740838.py:24: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/var/folders/5z/1fjczghs4x101sqrk1z655tc0000gn/T/ipykernel_18309/1906740838.py:25: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [11]:
# TODO: Split into compression utilization and decompression utilization
# TODO: Combine utilization data with throughput data

group_data_compress = group_data[group_data['category'] != CATEGORY_OTHER]

group_data_compress = group_data_compress.groupby('category').agg({'duration': 'mean',
                                                                   'utilization': 'mean',})

print(group_data_compress)

              duration  utilization
category                           
compress    15264181.0    41.787048
decompress   1562472.0    25.922448
